### Word Group In Youtube Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
shift = 0.3  # sentence time shift

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")


6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [6]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [7]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [8]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [9]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [10]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [11]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [12]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [19]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [20]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [21]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [22]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{file_ext}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

In [ ]:
df_sentence_ratio_result["start_time"] = df_sentence_ratio_result["start_time"].apply(lambda x: (x-shift))
df_sentence_ratio_result["end_time"] = df_sentence_ratio_result["end_time"].apply(lambda x: (x+shift))
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["start_time"] = df_sentence_ratio_result["start_time"].apply(lambda x: round(x))
df_sentence_ratio_result["end_time"] = df_sentence_ratio_result["end_time"].apply(lambda x: round(x))
df_sentence_ratio_result

In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{file_ext}_word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio